### <font color=red>如果下班了以后还想让服务器自己继续跑程序的话，可以在xshell中输入nohup python 2_2_data_process.py &就可以让服务器自动hang up自己跑

<font color=blue>nohup python 2_2_data_process.py &

In [1]:
import numpy as np
import pandas as pd
import warnings,os,time,gc,jieba
warnings.filterwarnings('ignore')

In [2]:
def random_shuffle(df):
    df=df.reset_index(drop=True)
    leng=len(df)
    indices=np.random.permutation(leng)
    df=df.iloc[indices,:]
    print(df.head())
    return df

In [3]:
#方法一：
#（1）对于非品牌字，随机选择与本品牌不重字的词组，组合成一个新的desc
#优点：增加了非固定词语组合的特征
#需要注意的是:categ的统一，即要选择categ一致的品牌替换才可以

#方法二：
#（1）找到那些不带categ的数据，然后将数据分成3类型，左中右
#（2）然后分别随机选择以后，将对应的品牌进行代替
#（3）然后对于这些新增的数据，全都放到train_0中，先出model,然后预测这些新增的样本的测试
#优点：更加符合实际业务中的platform_goods_name

In [4]:
train=pd.read_csv('train_ner_bieo.csv')
print(train.shape)
train[['id','desc','final_ner']].sample(3)

(4220070, 23)


,id,desc,final_ner
2149861,44737846730,下单减10元送5件套刷眉笔18色眼影盘多色哑光珠光学生抖音网红沙漠黄昏影楼大地色韩德国小众1...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3496462,69168184350,卡姿兰口红唇釉豆沙色唇彩唇蜜持久保湿不掉色不易沾杯润唇膏防水正V610枣泥粽,"BRD-B,BRD-M,BRD-E,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2138607,44533919986,DIOR迪奥滴管粉底液2015新款NUDEAIR俄罗斯专柜代购现货,"BRD-B,BRD-M,BRD-M,BRD-E,BRD-B,BRD-E,O,O,O,O,O,..."


### <font color=red>方法一

<font color=red>（1)提取出brands和nokey words

In [5]:
#查看位置分布
def get_position(v1,v2):
    v2=v2.split(',')
    if len(v1)==len(v2):
        if len(v1)<2:
            if v2[0]=='BRD-B':
                return 'head'
            else:
                return 'zero'
        else:
            if v2[0]=='BRD-B':
                return 'head'
            elif v2[-1]=='BRD-I':
                return 'tail'
            else:
                flag=''
                for i in range(1,len(v2)-1):
                    if v2[i]=='BRD-B':
                        flag='middle'
                if flag=='middle':
                    return 'middle'
                else:
                    return 'zero'

In [6]:
#查看位置分布
def get_position_bieo(v1,v2):
    v2=v2.split(',')
    if len(v1)==len(v2):
        if len(v1)<2:
            if v2[0]=='BRD-B':
                return 'head'
            else:
                return 'zero'
        else:
            if v2[0]=='BRD-B':
                return 'head'
            elif v2[-1]=='BRD-E':
                return 'tail'
            else:
                flag=''
                for i in range(1,len(v2)-1):
                    if v2[i]=='BRD-B':
                        flag='middle'
                if flag=='middle':
                    return 'middle'
                else:
                    return 'zero'

In [7]:
train['pos']=train[['desc','final_ner']].apply(lambda row:get_position_bieo(row[0],row[1]),raw=True,axis=1)
print(train.shape)
print(pd.isnull(train[['id','pos']]).sum())

(4220070, 24)
id     0
pos    0
dtype: int64


In [8]:
tmp=train.groupby('pos')['id'].count().reset_index(name='cnt')
tmp['ratio']=tmp['cnt']/np.sum(tmp['cnt'])
tmp.sort_values(by=['ratio'],ascending=False,inplace=True)
print(tmp.head())

      pos      cnt     ratio
0    head  1797389  0.425914
1  middle  1224828  0.290239
3    zero  1189561  0.281882
2    tail     8292  0.001965


In [ ]:
42.6, 29, 28

In [ ]:
# check=train[train['pos']=='middle'][['desc','final_ner']]
# check=check.sample(50).reset_index(drop=True)
# for i in range(50):
#     print(check['final_ner'][i])
#     print('-'*40)

In [71]:
def del_brand_jieba(v1,v2):
    lst_key=[]
    lst_nokey=[]
    
    lst=[]
    s=str(v1)
    ner=v2.split(',')
    if len(s)==len(ner):
        lst=[]
        for i in range(len(s)):
            word1=s[i]
            ner1=ner[i]
            if ner1=='O':
                lst_nokey.append(word1)
            elif ner1=='BRD-B':
                lst.append(word1)
            else:
                lst.append(word1)
        return 
    else:
        return 'wrong'

In [72]:
# #计算一个lst中出现的某一个value的次数
# from collections import Counter
# result = Counter(v2)
# print(result['BRD-B'])

In [10]:
v1='香港直邮雅诗兰黛YS花漾倾慕口红35ML'
v2='O,O,O,O,BRD-B,BRD-I,BRD-I,BRD-I,BRD-B,BRD-I,O,O,O,O,O,O,O,O,O,O'

def split_key_nokey(v1,v2):
    v2=v2.split(',')
    
    lst_nokey=[]
    lst_key=[]

    if len(v1)==len(v2):
        for i in range(len(v1)):
            w1=v1[i]
            n1=v2[i]
            if n1=='O':
                lst_nokey.append(w1)
            elif n1=='BRD-B':
                lst_key.append('#')
                lst_key.append(w1)
            else:
                lst_key.append(w1)
            
    a=''.join(lst_nokey)
    b=''.join(lst_key)
    lst=list(set(b.split('#')))
    if '' in lst:
        lst.remove('')
    return ','.join(lst),a

# keys,nokeys=split_key_nokey(v1,v2)
# print(keys)
# print(nokeys)

In [11]:
train['desc_key']=train[['desc','final_ner']].apply(lambda row:split_key_nokey(row[0],row[1])[0],raw=True,axis=1)

In [12]:
print(pd.isnull(train[['id','desc_key']]).sum())

id          0
desc_key    0
dtype: int64


In [13]:
print(len(train[train['desc_key']=='']))

1189550


In [14]:
train.loc[train['desc_key']!='','key_leng']=train[train['desc_key']!='']['desc_key'].apply(lambda x:len(x.split(',')))
train.loc[train['desc_key']=='','key_leng']=0

In [15]:
print(train['key_leng'].value_counts())

1.0    1933553
0.0    1189550
2.0    1075388
3.0      20722
4.0        855
5.0          2
Name: key_leng, dtype: int64


In [16]:
# check=train[train['key_leng']==5].reset_index(drop=True)
# print(check.head())

In [17]:
#品牌词频统计
s1=time.time()
brand_time_dict={}
train_del_null=train[train['desc_key']!=''][['id','desc_key']].drop_duplicates().reset_index(drop=True)
for i in range(len(train_del_null)):
    keys=train_del_null['desc_key'][i]
    keys=keys.split(',')
    for k in keys:
        if k not in brand_time_dict:
            brand_time_dict[k]=0
        brand_time_dict[k]+=1
print(len(brand_time_dict))
print('run time is:',(time.time()-s1)/60)
gc.collect()

28640
run time is: 2.749031440416972


26

In [18]:
brand_time_df=pd.DataFrame.from_dict(brand_time_dict,orient='index')
brand_time_df=brand_time_df.reset_index().rename(columns={'index':'brand',0:'frequency'})
print(brand_time_df.shape)
brand_time_df=brand_time_df.drop_duplicates().reset_index(drop=True)
print(brand_time_df.shape)
print(brand_time_df.head())

brand_time_df.to_csv('brand_frequency_bieo.csv',index=False,encoding='utf-8')
gc.collect()

(28640, 2)
(28640, 2)
        brand  frequency
0         934          5
1  PHILOSOPHY        138
2        1611          4
3       ROYAL        204
4       THREE       3122


44

In [19]:
print(brand_time_df['frequency'].quantile([0,0.25,0.5,0.75,1]))

0.00        1.0
0.25        1.0
0.50        3.0
0.75       13.0
1.00    96391.0
Name: frequency, dtype: float64


In [20]:
check=brand_time_df[brand_time_df['frequency']<3]
print(check.shape)
print(check['frequency'].min())

(13317, 2)
1


### <font color=red>先用那些样本量>=3的brand进行测试

In [21]:
choosed_brand=brand_time_df[brand_time_df['frequency']>=3]['brand'].unique().tolist()
print(len(choosed_brand))

15323


In [22]:
"""用以下品牌进行测试"""
test_brand=brand_time_df[brand_time_df['frequency']==3]
print(len(test_brand))
print(test_brand['brand'][:50].tolist())

2118
['23YEARSOLD', '嘉康利', 'CONY', '1831', 'YUZUN', 'ZNG', 'SWEETCOLO', 'CELLINA', 'SMOOTHSKIN', 'HRR', '瑷尔博士', 'DRALVA', '涵诚', 'TIVERTON', '自由膜法', '柏溪', '奈偲好优可', '红爱', '茶颜', '民联', '康先富的', '果佳果木', 'SACRUM', 'ISISPHARMA', '伊姿法玛', 'HAIRCUBE', '格格驾到', '初肌音', 'RYDEEN', 'FERNZ', '骄微尔', '逆光之谜', '安捷莉娜', '蕊珂', '美妮达', '黎芙莎', '医美霖', '诺思豪', '安婷', '乐宁清', 'ALAL', '肤见', 'NISEI', '云妞', 'AUX', '美颜嘎嘎', '美添沁', '莳语', '花茜茜', '爱柔肤']


In [23]:
train_df=train[train['desc_key'].str.contains('|'.join(choosed_brand))]
print(train.shape)
print(train_df.shape)

(4220070, 26)
(3027224, 26)


In [26]:
print(len(train_df)/len(train))

0.7173397597670181


In [24]:
train_df.to_csv('train_ner_3_bieo.csv',index=False,encoding='utf-8')

In [25]:
test=pd.read_csv('test_ner_bieo.csv')
print(test.shape)
test.to_csv('test_ner_3_bieo.csv',index=False,encoding='utf-8')
gc.collect()

(299402, 14)


0

#### <font color=red>首先,删除那些不是真实brand的brand<br>然后，对于那些样本量<3的brand做数据增强,最好不要用到categ，否则一定要注意替换的时候，一定要同categ下替换实体